In [1]:
using DifferentiableStateSpaceModels, LinearAlgebra, Optim, Turing, Zygote, BenchmarkTools, StatsPlots, TensorBoardLogger, Logging
Turing.setadbackend(:zygote)

using Turing: @addlogprob!
# Create models from modules and then solve
model_rbc = @include_example_module(Examples.rbc_observables_benchmark)

use_tensorboard = true # Toggle to turn off tensorboard logging
p_f = [0.7, 0.1, 0.01, sqrt(0.0001)]
# Generate artificial data
p = [0.4, 0.96]
sol = generate_perturbation(model_rbc, p; p_f)
T = 20
ϵ = [randn(model_rbc.n_ϵ) for _ in 1:T]
x0 = zeros(model_rbc.n_x)
fake_z = solve(sol, x0, (0, T), LTI(); noise = ϵ).z

## Turing model, Kalman filter
@model function rbc_kalman(z, m, p_f, cache)
    α ~ Uniform(0.2, 0.8)
    β ~ Uniform(0.5, 0.99)
    p = [α, β]
    sol = generate_perturbation(m, p; p_f, cache)
    if !(sol.retcode == :Success)
        @addlogprob! -Inf
        return
    end
    @addlogprob! solve(sol, sol.x_ergodic, (0, length(z)); observables = z).logpdf
end

turing_model = rbc_kalman(fake_z, model_rbc, p_f, allocate_cache(model_rbc))
n_samples = 20
n_adapts = 5
δ = 0.65
comment = "kalman-s$(n_samples)-a$(n_adapts)-δ$(δ)"
callback = make_turing_callback(model_rbc; comment, use_tensorboard)  # pass any TuringLogger options
chain = sample(turing_model, NUTS(n_adapts, δ), n_samples; progress = true, callback)
log_turing_results(chain, callback)
print("Sampling complete.  To view tensorboard logs, do `tensorboard --logdir runs`")

┌ Info: Found initial step size
│   ϵ = 0.4
└ @ Turing.Inference C:\Users\jesse\.julia\packages\Turing\dXPDh\src\inference\hmc.jl:188
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\jesse\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
Sampling:   8%|████                                     |  ETA: 0:00:04┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\jesse\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
Sampling:  68%|████████████████████████████             |  ETA: 0:00:02┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\jesse\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
Sampling: 100%|████████████████████████████████████████

Sampling complete.  To view tensorboard logs, do `tensorboard --logdir runs`